# **Entorno**

In [1]:
# https://www.properati.com.ar/data
#! wget -O ar_properties.csv.gz -q https://storage.googleapis.com/properati-data-public/ar_properties.csv.gz
#! gzip -d -f ./ar_properties.csv.gz

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import sklearn as sk
from sklearn import model_selection
from sklearn import ensemble
from sklearn import metrics

import datetime

import warnings
warnings.filterwarnings('ignore')

import re


In [3]:
pd.options.display.max_colwidth = -1

# Lectura de datos

In [4]:
df_properties = pd.read_csv('../ar_properties.csv', index_col="id")
df_apred = pd.read_csv('../a_predecir.csv', index_col="id")

In [5]:
df = df_properties.copy()
df_pred = df_apred.copy()

In [6]:
df.head()

,ad_type,start_date,end_date,created_on,lat,lon,l1,l2,l3,l4,...,bathrooms,surface_total,surface_covered,price,currency,price_period,title,description,property_type,operation_type
id,,,,,,,,,,,,,,,,,,,,,
DyVXfkpKygVBKuUk5olH+A==,Propiedad,2020-08-22,2020-09-03,2020-08-22,-34.407468,-58.957367,Argentina,Bs.As. G.B.A. Zona Norte,Pilar,NaN,...,NaN,133139.0,NaN,NaN,NaN,Mensual,VENTA - Lote Industrial 130.000m2 - Parque Industrial Pilar,"Lote (Nro.50) mide 133.139 m2\n<br>Valor: U$S (CONSULTAR)\n<br>\n<br>(Nuevo) PLAZA INDUSTRIAL PILAR (Anexo) del Parque Industrial Pilar.\n<br>El predio cuenta con 200 HA ofreciendo LOTES que van desde los 5.000 hasta 300.000m2.\n<br> FOS: 0.5 / FOT: 1\n<br> Zonificación Industrial (Categoría: 1, 2 y 3).\n<br> Desagues Pluviales\n<br> Tendido Red Eléctrica\n<br> Alumbrado Led de calles\n<br>Sobre Av. Arturo Frondizi entre (Calles 10 y 12), con acceso directo a la Au. Panamericana Ramal Pilar (Ruta 8, Km 61).\n<br>\n<br>CONSULTAR: Depósitos/Locales Llave en Mano (150 a los 1.000m2).\n<br>CONSULTAR: Disponibilidad y Financiamiento.<br><br>Martillero y Corredor Inmobiliario: <br>Walter Angel Tomé<br>CMCPSI 5200 <br>Todas las propiedades son tasadas y bajo normas establecidas en la ley publicamos las propiedades para su comercialización.<br>La intermediación y la conclusión de las operaciones son actividades exclusivas de este matriculado.",Lote,Venta
9naojilaMecJN4jlQiTkGg==,Propiedad,2020-08-22,2020-09-04,2020-08-22,-37.996039,-57.542509,Argentina,Buenos Aires Costa Atlántica,Mar del Plata,NaN,...,NaN,687.0,687.0,NaN,NaN,Mensual,Casa - Santa Cecilia,"Conjunto edilicio de casa y departamentos en block. Casona de 1930, restaurada, con vista al mar. A 30 mts de la costa, cerca de peatonal San Martín y otros puntos turísticos. <br><br>Apto para:<br>1. Vivienda familiar + uso de oficinas dentro del mismo edificio.<br>2. Uso profesional (consultorios, oficinas)<br>3. Alquiler de habitaciones tipo hostel o apart.<br>4. Apto para demoler y construir.<br><br>DETALLES:<br><br>P.B: Espacios comunes, 100 m2. Ingreso, cochera p/3 autos, lavadero, depósito, baño, parque de 134 m2 con parrilla, doble quincho, piscina para niños.<br><br>PRIMER PISO: Dos deptos con vista al mar (opción unificable para armar un único piso) con detalles de calidad, pinotea, piedra, etc.<br>Al frente, dos grandes amb con dormitorios, cocina com, gran living com/recepción, gran tarima de madera de primera, hogar y leñera, catarata decorativa, baño completo, balcón a la calle (89 m2)<br>Al parque, depto de living, cocina grande con balcón terraza (43 m2), dormitorio y baño.<br><br>SEGUNDO PISO: Cinco deptos completos (2 al fte y 3 laterales), todos externos, vista al mar, con baños, bauleras, cocinas y terraza.<br>Depto de 50 m2 (2 amb) + depto de 18 m2 (1 amb) + depto de 26 m2 (1 amb c/balcón) + depto de 33 m2 (2 amb) + depto de 49 m2 (2 amb) c/entrada independiente.<br>Los deptos de 2 amb pueden unificarse para armar uno mas grande, totalmente externo. <br><br>TERCER PISO: Un loft de 168 m2. Living com, 2 baños, sector dormitorios, 2 terrazas, entre piso con gimnasio o sala de estudio), cocina, lavadero, doble entrada, extraordinaria vista al mar y al parque.<br><br><br>SERVICIOS GENERALES:<br><br>Calefacción por radiadores, 2 calderas y 2 termotanques. Grupo electrógeno. Tanque de agua de 3500 lts + reserva de 600 lts. 2 bombas elevadoras y 1 de presión extra. Espacio ascensor-montacargas. Sistema anti incendio. Central telefónica. Alarma. TV satelital. Internet. Iluminación decorativa. <br><br>INSTALACIONES:<br><br>Totalmente equipado y amoblado. Todo en perfecto funcionamiento. Hornos. Anafes. Heladeras. TVs. Mobiliario. Vajilla. Accesorios (NO INCLUIDO EN EL PRECIO DE VENTA INICIAL)",Otro,Venta
tlCPRJPjoDEUzuuCelemAQ==,Propiedad,2020-08-22,2020-08-31,2020-08-22,-31.380187,-58.009182,Argentina,Entre Ríos,Concordia,NaN,...,1.0,80.0,80.0,NaN,NaN,Mensual,Casa - Concordia,"Casa de un dormitorio, cocina, baño, living comedor, lavadero, pati

In [7]:
df_pred.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1087 entries, 1 to 1087
Data columns (total 27 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ad_type          1087 non-null   object 
 1   start_date       1087 non-null   object 
 2   end_date         1087 non-null   object 
 3   created_on       1087 non-null   object 
 4   lat              1087 non-null   float64
 5   lon              1087 non-null   float64
 6   l1               1087 non-null   object 
 7   l2               1087 non-null   object 
 8   l3               1087 non-null   object 
 9   rooms            1083 non-null   float64
 10  bedrooms         958 non-null    float64
 11  bathrooms        1083 non-null   float64
 12  surface_total    1087 non-null   float64
 13  surface_covered  1087 non-null   float64
 14  price            0 non-null      float64
 15  currency         1087 non-null   object 
 16  price_period     0 non-null      float64
 17  title         

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000000 entries, DyVXfkpKygVBKuUk5olH+A== to rEynGyGrrsxNeG0ib5vUBA==
Data columns (total 24 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   ad_type          1000000 non-null  object 
 1   start_date       1000000 non-null  object 
 2   end_date         1000000 non-null  object 
 3   created_on       1000000 non-null  object 
 4   lat              894233 non-null   float64
 5   lon              894191 non-null   float64
 6   l1               1000000 non-null  object 
 7   l2               1000000 non-null  object 
 8   l3               965273 non-null   object 
 9   l4               306162 non-null   object 
 10  l5               5530 non-null     object 
 11  l6               0 non-null        float64
 12  rooms            714179 non-null   float64
 13  bedrooms         649933 non-null   float64
 14  bathrooms        765122 non-null   float64
 15  surface_total    477831 non-nul

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000000 entries, DyVXfkpKygVBKuUk5olH+A== to rEynGyGrrsxNeG0ib5vUBA==
Data columns (total 24 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   ad_type          1000000 non-null  object 
 1   start_date       1000000 non-null  object 
 2   end_date         1000000 non-null  object 
 3   created_on       1000000 non-null  object 
 4   lat              894233 non-null   float64
 5   lon              894191 non-null   float64
 6   l1               1000000 non-null  object 
 7   l2               1000000 non-null  object 
 8   l3               965273 non-null   object 
 9   l4               306162 non-null   object 
 10  l5               5530 non-null     object 
 11  l6               0 non-null        float64
 12  rooms            714179 non-null   float64
 13  bedrooms         649933 non-null   float64
 14  bathrooms        765122 non-null   float64
 15  surface_total    477831 non-nul

# Tratamiento de nulls y outliers

In [10]:
# Me quedo solo con CABA
df = df.loc[df["l1"] == "Argentina"]
df = df.loc[df["l2"] == "Capital Federal"]

## Longitud y Latitud

In [11]:
df.dropna(subset=["lon"],inplace=True)
df.dropna(subset=["lat"],inplace=True)

In [12]:
lon_min_max = (58.532674897063565, 58.338011418419356 ) #Limites Este/Oeste de CABA
lat_min_max = (34.7058110012102, 34.531639) #Limites Norte/Sur de CABA

Primero acomodo lon y lat de los datos a predecir, como no podemos dropear los na y outliers les vamos a asignar el valor que corresponda.

In [13]:
df_pred[["lat","lon"]] = df_pred[["lon","lat"]] # Los invertimos ya que se encontraban al revez

In [14]:
# Establezco el valor absoluto de los campos
df_pred["lon"] = abs(df_pred["lon"]) 
df_pred["lat"] = abs(df_pred["lat"]) 

In [15]:
# Lo que este fuera de la longitu y latitud maxima de CABA es outlier
df_pred["lat_outlier"] = df_pred.lat.apply(lambda x: 0 if (x < lat_min_max[0]) & (x > lat_min_max[1]) else 1)
df_pred["lon_outlier"] = df_pred.lon.apply(lambda x: 0 if (x < lon_min_max[0]) & (x > lon_min_max[1]) else 1)

In [16]:
#Buscamos la longitud y latitud de los outliers según lo que vemos en el titulo de la publicación
df_pred.loc[422,"lat"] = 34.628212
df_pred.loc[873,"lat"] = 34.557626
df_pred.loc[1055,"lat"] = 34.601223
df_pred.loc[1057,"lat"] = 34.557626
df_pred.loc[422,"lon"] = 58.426367
df_pred.loc[873,"lon"] = 58.484324
df_pred.loc[1055,"lon"] = 58.376599
df_pred.loc[1057,"lon"] = 58.484324

df_pred.loc[1057,"lat"] = 34.580921

Ahora dropeo todos los outliers del set de entrenamiento

In [17]:
df["lon"] = abs(df["lon"]) 
df["lat"] = abs(df["lat"]) 

In [18]:
df["lat_outlier"] = df.lat.apply(lambda x: 0 if (x < lat_min_max[0]) & (x > lat_min_max[1]) else 1)
df["lon_outlier"] = df.lon.apply(lambda x: 0 if (x < lon_min_max[0]) & (x > lon_min_max[1]) else 1)

In [19]:
#Dropeo todos los otliers
df.drop(df.loc[(df["lat_outlier"] == 1) | (df["lon_outlier"] == 1)].index, inplace = True)

#Drepeo las dos columnas de outliers
df.drop(columns=["lat_outlier","lon_outlier"], inplace = True)
df_pred.drop(columns=["lat_outlier","lon_outlier"], inplace = True)

## Barrios

In [20]:
df_l3 = df[["lon","lat","l3"]]

df_test_l3 = df_l3.loc[df_l3["l3"].isnull()]

df_train_l3 = df_l3.loc[df_l3["l3"].notnull()]

In [21]:
X_l3 = df_train_l3[df_train_l3.columns.drop('l3')]
y_l3 = df_train_l3['l3']

# Partimos en entrenamiento y prueba 

for i in [1,3,5,7,9,11]:
    # Creamos el objeto del modelo
    neigh = sk.neighbors.KNeighborsClassifier(n_neighbors=i)
    
    kf = sk.model_selection.KFold(n_splits=10, shuffle=True, random_state=42)
    
    scores = []
    
    for train, test in kf.split(X_l3):
        
        X_train, X_test, y_train, y_test = X_l3.iloc[train], X_l3.iloc[test], y_l3[train], y_l3[test]
    
        # Entrenamos el modelo
        neigh.fit(X_train, y_train)
    
        # Predecimos
        y_pred_l3 = neigh.predict(X_test)
    
        # Medimos la performance de la predicción
        score = sk.metrics.accuracy_score(y_test, y_pred_l3)
        scores.append(score)
        #print(score)
    print(f"n_{i} {np.mean(scores)}")


n_1 0.8281104459358722
n_3 0.8271963715414128
n_5 0.8290366500360132
n_7 0.8296473923071274
n_9 0.8290892372316154
n_11 0.8285391842217147


In [22]:
X_pred_l3 = df_test_l3[df_test_l3.columns.drop('l3')]

# Entrenamiento
neigh = sk.neighbors.KNeighborsClassifier(n_neighbors=7)

# Entrenamos el modelo
neigh.fit(X_train, y_train)

# predicción
df_test_l3["l3"] = neigh.predict(X_pred_l3)

In [23]:
df["l3"].loc[df["l3"].isnull()] = df_test_l3["l3"]

## Precio y moneda

In [24]:
df.dropna(subset=["price"], inplace = True)

df.dropna(subset=["currency"], inplace = True)

In [25]:
df = df.loc[df["currency"] == "USD"]

df.drop(columns = ["currency"], inplace = True )

## Tipo de propiedad

In [26]:
df["property_type_check"] = df.property_type.apply(lambda x: 1 if x in ("Departamento","PH") else 0)

df = df.loc[df["property_type_check"] == 1]

df.drop(columns = "property_type_check", inplace = True)

## Habitaciones y Superficie

Si no tiene ninguna de las 5 variables lo elimino

In [27]:
df.drop(df.loc[df["rooms"].isnull() & df["bedrooms"].isnull() & df["bathrooms"].isnull() & df["surface_total"].isnull() & df["surface_covered"].isnull()].index, inplace = True)

In [28]:
df_pred["rooms"].fillna(0, inplace = True)
df_pred["bedrooms"].fillna(0, inplace = True)
df_pred["bathrooms"].fillna(0, inplace = True)

In [29]:
df["rooms"].fillna(0, inplace = True)
df["bedrooms"].fillna(0, inplace = True)
df["bathrooms"].fillna(0, inplace = True)

In [30]:
df["bedrooms"].loc[df["bedrooms"] < 0 ] = abs(df["bedrooms"].loc[df["bedrooms"] < 0 ])

Detectamos que hay superficies en negativo, pasamos los valores a absolutos.

En caso de que haya superficie cubierta pero no total, asigno el valor de superficie cubierta.

En el caso inverso le asigno 0 a superficie cubierta, ya que lo considero como un lote sin construir.

In [31]:
df["surface_total"].loc[df["surface_total"] < 0 ] = abs(df["surface_total"].loc[df["surface_total"] < 0 ])

df["surface_covered"].loc[df["surface_covered"] < 0 ] = abs(df["surface_covered"].loc[df["surface_covered"] < 0 ])

df["surface_total"].loc[df["surface_total"].isnull() & df["surface_covered"] > 0] = df["surface_covered"].loc[df["surface_total"].isnull() & df["surface_covered"] > 0]

df["surface_covered"].loc[df["surface_covered"].isnull() & df["surface_total"] > 0] = 0


In [32]:
df["surface_total"].fillna(0, inplace = True)
df["surface_covered"].fillna(0, inplace = True)

In [33]:
df_pred["surface_total"].fillna(0, inplace = True)
df_pred["surface_covered"].fillna(0, inplace = True)

## Elimino las columnas que no utilizo

In [34]:
df.drop(columns = ["ad_type","l4","l5", "l6", "created_on","start_date", "end_date","l1","l2","price_period","operation_type"], inplace = True )

In [35]:
df.drop(columns = ["title","description"], inplace = True)

In [36]:
df.drop_duplicates(inplace = True)

In [37]:
df_pred.drop(columns = ["ad_type","l4","l5", "l6", "created_on"], inplace = True )
df_pred.drop(columns = ["l1","l2"], inplace = True )
df_pred.drop(columns = ["currency"], inplace = True )
df_pred.drop(columns = ["price_period"], inplace = True)
df_pred.drop(columns = "operation_type", inplace = True)
df_pred.drop(columns = ["start_date","end_date","services","additionals"], inplace = True)
df_pred.drop(columns = ["title","description","details"], inplace = True)

# Feature Engineering

Uno los dos data frames

In [38]:
df_total = pd.concat([df,df_pred], axis=0)

## Normalizamos min_max

In [39]:
def norm_min_max(min, max, value):
    return (value - min) / max

In [40]:
df_total["n_rooms"] = df_total.rooms.apply(lambda x: norm_min_max(df_total.rooms.min(),df_total.rooms.max(),x))

In [41]:
df_total["n_bedrooms"] = df_total.bedrooms.apply(lambda x: norm_min_max(df_total.bedrooms.min(),df.bedrooms.max(),x))

In [42]:
df_total["n_bathrooms"] = df_total.bathrooms.apply(lambda x: norm_min_max(df_total.bathrooms.min(),df.bathrooms.max(),x))

In [43]:
df_total["n_surface_covered"] = df_total.surface_covered.apply(lambda x: norm_min_max(df_total.surface_covered.min(),df.surface_covered.max(),x))

In [44]:
df_total["n_surface_total"] = df_total.surface_total.apply(lambda x: norm_min_max(df_total.surface_total.min(),df.surface_total.max(),x))

## Dummies

In [50]:
df_total.select_dtypes(include = "object").columns

Index(['l3', 'property_type'], dtype='object')

In [51]:
df_total = pd.get_dummies(df_total, columns = ["l3", "property_type"])

Dropeamos las columnas que normalizamos

In [52]:
df_total.drop(columns = ["rooms","bedrooms","bathrooms","surface_covered","surface_total"], inplace = True)

## Dividimos los DF

In [53]:
df = df_total[df_total.index.isin(df.index)]
df_pred = df_total[df_total.index.isin(df_pred.index)]

# Modelización

In [54]:
X = df[df.columns.drop('price')]
y = df['price']

## GBM

In [55]:

for n_estimators in [100, 500, 700]:
    for max_depth in [3, 5, 7]:     


        gbm = sk.ensemble.GradientBoostingRegressor(n_estimators=n_estimators, max_depth=max_depth,criterion="squared_error" ,random_state=42)

        scores = []
        kf = sk.model_selection.KFold(n_splits=5, shuffle=True, random_state=42)
        for train, test in kf.split(X):
            X_train, X_test, y_train, y_test = X.iloc[train], X.iloc[test], y[train], y[test]
        
            # Entrenamos el modelo
            gbm.fit(X_train, y_train)
        
            # Predecimos
            y_pred = gbm.predict(X_test)
        
            # Medimos la performance de la predicción
            score = sk.metrics.mean_squared_error(y_test, y_pred,squared=False)
            scores.append(score)
        print(f"n_{n_estimators} | max_depth:{max_depth} | mean: {np.mean(scores)} | sd: {np.std(scores)}")
        
    print("------------------------------------------------------------------------------------")

## Predicción para Kaggle


In [56]:
# quedate con las mismas columnas que ar_properties
X_pred = df_pred[X.columns]

In [58]:
# los mejores hiperparámetros encontrados antes
n_estimators = 500
max_depth = 55

# Entrenamiento
gbm = sk.ensemble.GradientBoostingRegressor(n_estimators=, max_depth= ,criterion="squared_error" ,random_state=42)

# Entrenamos el modelo
gbm.fit(X, y)

# predicción
df_pred["price"] = gbm.predict(X_pred)

In [59]:
# Grabo la solución
df_pred[["price"]].to_csv("solucion.csv")